In [ ]:
sessionInfo()

In [ ]:
# Prepare tx2gene
library(readr)

filename_tx2gene <- 'XENLA_XB201901_tx_Annot.tx2gene'
tx2gene <- read_tsv(filename_tx2gene, col_names=FALSE)
head(tx2gene)

In [ ]:
# Load quantification data
library(tximport)
library(limma)
library(edgeR)

# Uncomment and change the filename below
#filename_tbl <- 'KimHY202001n_XENLAtx.conf'
file.exists(filename_tbl)

samples <- read.table(filename_tbl, header=TRUE)
files <- trimws(samples$Filename)
names(files) <- samples$SampleName
files
all(file.exists(files))

txi <- tximport(files, type = "salmon", tx2gene = tx2gene, countsFromAbundance = "lengthScaledTPM")
y <- DGEList(txi$counts)
# filtering
keep <- filterByExpr(y)
y <- y[keep, ]

In [ ]:
# Boxplot and hierarchical clustering with raw data
options(repr.plot.width=12, repr.plot.height=6)

boxplot(log(txi$abundance+0.001, base=2), las=2)
dist_corr <- dist(1-cor(as.matrix(txi$abundance), method='spearman'))
plot(hclust(dist_corr))

In [ ]:
# Boxplot and hierarchical clustering with filtered data
options(repr.plot.width=12, repr.plot.height=6)

boxplot(log(as.matrix(y)+0.001, base=2), las=2, outline=FALSE)
dist_corr <- dist(1-cor(as.matrix(y), method='spearman'))
plot(hclust(dist_corr))

In [ ]:
# Pair plot
library(ggplot2)

#pairs(txi$abundance+0.001, log="xy")
#install.packages("psych")
library(psych)
pairs.panels(log(txi$abundance+0.001),
             method = "spearman", # correlation method
             hist.col = "#00AFBB",
             density = TRUE,  # show density plots
             ellipses = TRUE # show correlation ellipses
             )

In [ ]:
# Perform voom
sampleTable <- data.frame(condition = factor(rep(c("A", "B"), each = 3)))
rownames(sampleTable) <- colnames(txi$counts)

y <- calcNormFactors(y)
design <- model.matrix(~condition, data = sampleTable)
v <- voom(y, design)